In [19]:
import findspark
findspark.init()
import pyspark
from pyspark.sql import *
from pyspark.sql.functions import when, lit

In [3]:
sc = pyspark.SparkContext(appName="Pi")
spark = SparkSession(sc)

23/06/25 16:56:37 WARN Utils: Your hostname, DSaDBA resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
23/06/25 16:56:37 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/06/25 16:56:38 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
banking_df = spark.read.csv("data.csv", inferSchema = True, header = True)

In [6]:
banking_df.head()

Row(ID=1, Age=25, Experience=1, Income=49, ZIP Code=91107, Family=4, CCAvg=1.6, Education=1, Mortgage=0, Personal Loan=0, Securities Account=1, CD Account=0, Online=0, CreditCard=0)

In [7]:
banking_df.printSchema()

root
 |-- ID: integer (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Income: integer (nullable = true)
 |-- ZIP Code: integer (nullable = true)
 |-- Family: integer (nullable = true)
 |-- CCAvg: double (nullable = true)
 |-- Education: integer (nullable = true)
 |-- Mortgage: integer (nullable = true)
 |-- Personal Loan: integer (nullable = true)
 |-- Securities Account: integer (nullable = true)
 |-- CD Account: integer (nullable = true)
 |-- Online: integer (nullable = true)
 |-- CreditCard: integer (nullable = true)



In [ ]:
## boomers are older than 59 this year (2023) 59-77 years old
## generationX are older than 44 this year and less than 59 44-58 years old
## milennials are older than 29 this year 29-43 years old
## generationZ are older than 11 this year 11-28 years old


In [8]:
boomers = banking_df.filter(banking_df.Age>58)

In [11]:
generationX = banking_df.filter((banking_df.Age>=44)&(banking_df.Age<59))

In [14]:
millenials=banking_df.filter((banking_df.Age>=29)&(banking_df.Age<44))


In [15]:
generationZ=banking_df.filter((banking_df.Age>=11)&(banking_df.Age<29))

In [36]:
banking_df = banking_df.withColumn("Generation",
    when((banking_df.Age <= 28), lit("GenerationZ"))
    .when((banking_df.Age > 28) & (banking_df.Age <= 44), lit("Millennials"))
    .when((banking_df.Age > 44) & (banking_df.Age <= 59), lit("GenerationX"))
    .otherwise(lit("Boomers"))
)

In [37]:
banking_df.columns

['ID',
 'Age',
 'Experience',
 'Income',
 'ZIP Code',
 'Family',
 'CCAvg',
 'Education',
 'Mortgage',
 'Personal Loan',
 'Securities Account',
 'CD Account',
 'Online',
 'CreditCard',
 'Generation']

In [38]:
generation_df = banking_df.groupby("Generation")
generation_df.count().show()

+-----------+-----+
| Generation|count|
+-----------+-----+
|GenerationX| 1934|
|Millennials| 2027|
|GenerationZ|  365|
|    Boomers|  674|
+-----------+-----+



In [41]:
generation_df